In [ ]:
%%javascript
IPython.OutputArea.auto_scroll_threshold = 9999;

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [ ]:
#***********************************************************************
# WKP viewer: HeatMap beta 0.1
# 
# Copyright (c) 2019, Cor Berrevoets, registax@gmail.com
#
# Permission is hereby granted, free of charge, to any person obtaining a copy
# of this software and associated documentation files (the "Software"), to deal
# in the Software without restriction, including without limitation the rights
# to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
# copies of the Software, and to permit persons to whom the Software is
# furnished to do so, subject to the following conditions:
#
# The above copyright notice, development funding notice, and this permission
# notice shall be included in all copies or substantial portions of the Software.
#
# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
# AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
# OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN
# THE SOFTWARE.
# 
#*********************************************************************/


#default framework for applications
import param, panel as pn, holoviews as hv, numpy as np, pandas as pd, hvplot.pandas
hv.extension('bokeh')
renderer = hv.renderer('bokeh')
pn.extension()
#output_notebook()
#Read datasets
# #use temp storage to experiment
#sub1 = pd.read_hdf('data.h5',key='subs')
#read all oordelen 2009, 2014,2015,2016,2017,2018
sub1=pd.read_csv('oordeel.csv.gz')

#         0         1               2            3         4            5       6       7       8        9         10
colors=['red'   ,'gold'      , 'yellow', 'lightgreen', 'blue', 'white', 'white','lightsalmon','lightskyblue','red'   , 'blue']
omap= { 'oordeel':{ 'slecht':0,'ontoereikend':1,'matig':2,'goed':3,'zeer goed':4,'voldoet niet':9,'voldoet':10} }
#projectie 2018
localdir='datasets/'
datafiles= ['4.monitoringprogramma_owl_20181016.csv.gz']
locs=pd.DataFrame()
for index in range(len(datafiles)):
    locd=pd.read_csv(localdir+datafiles[index], sep=';', decimal='.', dtype=object, encoding='latin1')
    locd.columns = locd.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('.', '_').str.replace(')', '')
    locs=locs.append(locd)
locd=locs.copy()    

#combine oordelen and projectie
merged=pd.merge(sub1,locd,left_on=['waterlichaam_identificatie','chemischestof_omschrijving'],
                right_on=['representatief_waterlichaam_code','parameter_omschrijving'], how='left')

#force rapportagejaar to integervalue
merged.rapportagejaar=merged.rapportagejaar.astype(int)

#remove biological oordeel, select 2018 and mark projections 
merged=merged.dropna(subset=['chemischestof_omschrijving'])
#merged=merged.query('rapportagejaar=="2018"')
merged['projectie']=merged['waterlichaam_identificatie']!=merged['ligt_in_waterlichaam_code'] 

#set lists for selectors
startup=True
class dataPlot(param.Parameterized):
    
    #definition of widget-based-parameters
    beheerder = param.ObjectSelector(default="Waterschap Scheldestromen", objects=sorted(list(merged.waterbeheerder_omschrijving.unique())))
    stroomgebied = param.ObjectSelector(default='all', objects=['all']+list(merged.stroomgebieddistrict_code.unique()))
    jaarselector = param.ObjectSelector(default=2018, objects=['all']+sorted(list(merged.rapportagejaar.unique())))
    methode= param.ObjectSelector(default='max', objects=['min','max', 'last'] )
    #dataselection 
    def get_data(self):
        sub1=merged.copy()
        if self.jaarselector!='all':
            sub1=sub1.query('rapportagejaar==@self.jaarselector')   
        if self.stroomgebied!='all':
            sub1=sub1.query('stroomgebieddistrict_code==@self.stroomgebied')   
        else:
            sub1=sub1.query('waterbeheerder_omschrijving==@self.beheerder')
        #converteer oordeel naar numerieke waarde
        sub1=sub1.replace(omap)
        
        #split dataset by projectie
        #projectie is only available for 2018 data
        if self.jaarselector==2018:
            split1, split2 = [x for _, x in sub1.groupby(sub1['projectie'])]
            sub3=split2.query('oordeel<8').copy() #use this only for chemicals so split others to sub3 /KEEP
            sub2=split2.query('oordeel>8').copy() #the chemicals, replace oordeel where projected with oordeel-2
            sub2['oordeel'] = sub2['oordeel']-2 
            sub2=pd.concat([sub2,sub3,split1]).drop_duplicates() #recombine dataset and remove duplicates
        else:
            sub2=sub1.copy()
            
        #create dataset for heatmap, select max or min or last per wl, partially projected will become NOT projected 
        #sub2=sub2.sort_values(['waterlichaam_identificatie','chemischestof_omschrijving','rapportagejaar'])
        sub2=sub2.sort_values(['waterlichaam_identificatie','chemischestof_omschrijving','rapportagejaar'])
        if self.methode=='last':  
            subs=sub2.groupby(['waterlichaam_identificatie','waterlichaam_type','chemischestof_omschrijving',
                           'stroomgebieddistrict_code','waterbeheerder_omschrijving']).last()
        else:
            subs=sub2.groupby(['waterlichaam_identificatie','waterlichaam_type','chemischestof_omschrijving',
                               'stroomgebieddistrict_code','waterbeheerder_omschrijving'])
            
        if self.methode=='max':  
            sub2=pd.DataFrame(subs['oordeel'].max()).reset_index()
        if self.methode=='min':  
            sub2=pd.DataFrame(subs['oordeel'].min()).reset_index()
            
        return sub2

    def save(self):
        pass
    
    @param.depends('beheerder','stroomgebied','jaarselector','methode', watch=True) #watch reacts to any change of paramA or param1
    def create_plot(self):
        df = self.get_data()
        #get the last valid oordeel
        if self.methode=='last':
            df=df.groupby(['waterlichaam_identificatie','waterlichaam_type','chemischestof_omschrijving',
                               'stroomgebieddistrict_code','waterbeheerder_omschrijving']).last().reset_index()
        
        df=df.sort_values(by=['waterlichaam_identificatie','chemischestof_omschrijving'])
        low=df['oordeel'].min() #find lowest value to allow slicing the cmap-dictionary
        cats=df.chemischestof_omschrijving.unique() #coun
        pheight=cats.shape[0]*30+50
        if startup:
            pheight=1500
        #TODO create link to np function instead of copying heatmap routine    
        p=df.hvplot.heatmap(colorbar=False,ylabel='',rot=90,flip_yaxis=True, width=1000, height=pheight,
                            x='waterlichaam_identificatie' , y='chemischestof_omschrijving',
                            C='oordeel',   hover_cols=['waterbeheerder_omschrijving','stroomgebieddistrict_code'],
                            cmap=colors[low:] ).opts(xaxis='top',yaxis='right', xlabel='', )
        
        #renderer.save(p, 'test')
        return p

dataplot = dataPlot(name='')

#external widget
button = pn.widgets.Button(name='Click me')
def on_click(event):
    viewplot.paramA='d'
button.param.watch(on_click, 'clicks')

#paramcontrols can be called like this to allow carefull building of the gui
plt=pn.panel(dataplot.create_plot)
right = pn.panel(dataplot, parameters=['beheerder'], width=150)
left = pn.panel(dataplot, parameters=['stroomgebied'], width=150)
left1 = pn.panel(dataplot, parameters=['jaarselector'], width=150)
right1 = pn.panel(dataplot, parameters=['methode'], width=150)
desc = pn.pane.HTML("""
    <br><h3>Monitoringprogramma en oordelen</h3><br>
    Overzicht van monitoring van chemie, gebaseerd op WKP gegevens uit 2009,2014-2018.
    Het oordeel voor chemische stoffen is rood/blauw indien gemeten, lichtere tinten rood/blauw 
    geven aan waar gegevens <b>in 2018</b> uitsluitend op projectie zijn gebaseerd.
    """, width=450)

# pn.Column(desc,
#           pn.Row(left,right), 
#           plt).servable()
pn.Column(pn.Row(desc,pn.Column(pn.Spacer(height=30),pn.Row(pn.Spacer(width=50), pn.Column(left,left1), pn.Column(right,right1 ) ))),
          pn.Spacer(height=20),
          plt ).servable()